In [1]:
import paddle
paddle.utils.run_check()

Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


In [1]:
!python tools/train.py -c configs/ppyoloe/ppyoloe_crn_s_300e_coco.yml --eval --amp

^C


In [1]:
import os
import json
import cv2
import random
import time
from PIL import Image
 
 
 
def yolotxt2cocojson(save_dir,class_txt_path,label_dir,img_dir):
    with open(class_txt_path,'r') as fr:                               #打开并读取类别文件
        lines1=fr.readlines()
    # print(lines1)
    categories=[]                                                                 #存储类别的列表
    for j,label in enumerate(lines1):
        label=label.strip()
        categories.append({'id':j+1,'name':label,'supercategory':'None'})         #将类别信息添加到categories中
    # print(categories)
 
    write_json_context=dict()                                                      #写入.json文件的大字典
    write_json_context['info']= {'description': '', 'url': '', 'version': '', 'year': 2024, 'contributor': 'liang.yingyi', 'date_created': '2024-01-22'}
    write_json_context['licenses']=[{'id':1,'name':None,'url':None}]
    write_json_context['categories']=categories
    write_json_context['images']=[]
    write_json_context['annotations']=[]
 
    #接下来的代码主要添加'images'和'annotations'的key值
    img_list=os.listdir(img_dir)                                           #遍历该文件夹下的所有文件，并将所有文件名添加到列表中
    for i,img_name in enumerate(img_list):
        img_path = os.path.join(img_dir,img_name)                             #获取图片的绝对路径
        image = Image.open(img_path)                                               #读取图片，然后获取图片的宽和高
        W, H = image.size
        # print(img_name)
        # print(type(img_name))
 
        img_context={}                                                              #使用一个字典存储该图片信息
        #img_name=os.path.basename(img_path)                                       #返回path最后的文件名。如果path以/或\结尾，那么就会返回空值
        img_context['file_name']=img_name
        # print(img_context['file_name'])
        img_context['height']=H
        img_context['width']=W
        img_context['date_captured']='2024-01-22'
        img_context['id']=i                                                         #该图片的id
        img_context['license']=1
        img_context['color_url']=''
        img_context['flickr_url']=''
        write_json_context['images'].append(img_context)                            #将该图片信息添加到'image'列表中
        # print(img_name)
 
        label_name = img_name.split('.')[0] +'.txt'                                          #获取该图片对应的txt文件
        with open(os.path.join(label_dir,label_name),'r') as fr:
            lines=fr.readlines()                                                   #读取txt文件的每一行数据，lines2是一个列表，包含了一个图片的所有标注信息
        for j,line in enumerate(lines):
 
            bbox_dict = {}                                                          #将每一个bounding box信息存储在该字典中
            # line = line.strip().split()
            # print(line.strip().split(' '))
 
            class_id,x,y,w,h=line.strip().split(' ')                                          #获取每一个标注框的详细信息
            class_id,x, y, w, h = int(class_id), float(x), float(y), float(w), float(h)       #将字符串类型转为可计算的int和float类型
 
            xmin=(x-w/2)*W                                                                    #坐标转换
            ymin=(y-h/2)*H
            xmax=(x+w/2)*W
            ymax=(y+h/2)*H
            w=w*W
            h=h*H
 
            bbox_dict['id']=i*10000+j                                                         #bounding box的坐标信息
            bbox_dict['image_id']=i
            bbox_dict['category_id']=class_id+1                                               #注意目标类别要加一
            bbox_dict['iscrowd']=0
            height,width=abs(ymax-ymin),abs(xmax-xmin)
            bbox_dict['area']=height*width
            bbox_dict['bbox']=[xmin,ymin,w,h]
            bbox_dict['segmentation']=[[xmin,ymin,xmax,ymin,xmax,ymax,xmin,ymax]]
            write_json_context['annotations'].append(bbox_dict)                               #将每一个由字典存储的bounding box信息添加到'annotations'列表中
    save_name = img_dir.split('/')[-1]
    name = os.path.join(save_dir,'{}.json'.format(save_name))
    with open(name,'w') as fw:                                                                #将字典信息写入.json文件中
        json.dump(write_json_context,fw,indent=2,ensure_ascii=False)                          #加后缀ensure_ascii=False放置写入后中文乱码
 
def the_number_of_file_in_dir(img_dir,label_dir):
    #检测两个文件夹下的文件数量
    lable_list = os.listdir(label_dir)
    img_list =  os.listdir(img_dir)
    print(len(lable_list))
    print(len(img_list))
 
def creat_train_file(save_dir,class_txt_path,label_dir,img_dir):
    #生成训练集文件
    save_dir = os.path.join(save_dir,'train')
    label_dir = os.path.join(label_dir,'train')
    img_dir = os.path.join(img_dir,'train')
    # the_number_of_file_in_dir(img_dir,label_dir)
    os.makedirs(save_dir,exist_ok=True)
    yolotxt2cocojson(save_dir,class_txt_path,label_dir,img_dir)
 
def creat_val_file(save_dir,class_txt_path,label_dir,img_dir):
    #生成验证集文件
    save_dir = os.path.join(save_dir,'val')
    label_dir = os.path.join(label_dir,'val')
    img_dir = os.path.join(img_dir,'val')
    # the_number_of_file_in_dir(img_dir,label_dir)
    os.makedirs(save_dir,exist_ok=True)
    yolotxt2cocojson(save_dir,class_txt_path,label_dir,img_dir)
 
def creat_test_file(save_dir,class_txt_path,label_dir,img_dir):
    #生成测试集文件
    save_dir = os.path.join(save_dir,'test')
    label_dir = os.path.join(label_dir,'test')
    img_dir = os.path.join(img_dir,'test')
    # the_number_of_file_in_dir(img_dir,label_dir)
    os.makedirs(save_dir,exist_ok=True)
    yolotxt2cocojson(save_dir,class_txt_path,label_dir,img_dir)
 
 
if __name__ == '__main__':
    save_dir='D:\Documents\liangyy\Python\PaddleDetection.lyy\dataset\coco-pig'   #目标文件夹，用来保存生成的coco格式文件
    class_txt_path=os.path.join(save_dir, 'labels\classes.txt')     #类别文件，一行一个类,自行创建
    label_dir=os.path.join(save_dir, 'labels')  #yolo格式标签所在文件夹
    img_dir=os.path.join(save_dir, 'images')    #图片所在文件夹
    creat_train_file(save_dir,class_txt_path,label_dir,img_dir)
    creat_val_file(save_dir,class_txt_path,label_dir,img_dir)